In [1]:
import os
from urllib.parse import urlparse
from collections import defaultdict

import pandas as pd

from db.mongo import MyMongo

In [2]:
with MyMongo() as db:
    url_script = db.find('public_website', 'c1_public_js_list')

without_js_script = []

for item in url_script:
    item.pop('jsScript')
    without_js_script.append(item)

with MyMongo() as db:
    db.delete_and_insert('public_website', 'c2_js_files', without_js_script)

domain_unique_js_file = defaultdict(set)

for item in without_js_script:
    scheme = urlparse(item['url'])[0]
    domain = item['netLoc']
    scheme_domain = scheme + "://" + domain
    if item['jsFile']:
        js_files = set(item['jsFile'])
        domain_unique_js_file[scheme_domain] = domain_unique_js_file[scheme_domain].union(js_files)

df_data = []
for domain, unique_files in domain_unique_js_file.items():
    for file in unique_files:
        df_data.append((domain, file))

df_files_per_domain = pd.DataFrame(columns=['netLoc', 'jsFile'], data=df_data)
# df_files_per_domain['path'] = os.path.join(df_files_per_domain['netLoc'], df_files_per_domain['jsFile'])

with MyMongo() as db:
    db.delete_and_insert_df('public_website', 'c3_js_files_per_domain', df_files_per_domain)

<--Mongo Connected.
Mongo Connection Closed.-->
<--Mongo Connected.
{}
Deleted rows: 366
Inserted rows: 381
Mongo Connection Closed.-->
<--Mongo Connected.
{}
Deleted rows: 1811
Inserted rows: 1959
Mongo Connection Closed.-->


In [3]:
url = 'https://www.google'